In [ ]:
import io
import numpy as np
import sys
from gym.envs.toy_text import discrete
import pprint

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

pp = pprint.PrettyPrinter(indent=4)

class GridworldEnv(discrete.DiscreteEnv):
    """
    Grid World environment from Sutton's Reinforcement Learning book chapter 4.
    You are an agent on an MxN grid and your goal is to reach the terminal
    state at the top left or the bottom right corner.
    For example, a 4x4 grid looks as follows:
    T  o  o  o
    o  x  o  o
    o  o  o  o
    o  o  o  T
    x is your position and T are the two terminal states.
    You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3).
    Actions going off the edge leave you in your current state.
    You receive a reward of -1 at each step until you reach a terminal state.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, shape=[4,4]):
        if not isinstance(shape, (list, tuple)) or not len(shape) == 2:
            raise ValueError('shape argument must be a list/tuple of length 2')

        self.shape = shape

        nS = np.prod(shape)
        nA = 4

        MAX_Y = shape[0]
        MAX_X = shape[1]

        P = {}
        grid = np.arange(nS).reshape(shape)
        it = np.nditer(grid, flags=['multi_index'])

        while not it.finished:
            s = it.iterindex
            y, x = it.multi_index

            # P[s][a] = (prob, next_state, reward, is_done)
            P[s] = {a : [] for a in range(nA)}

            is_done = lambda s: s == 0 or s == (nS - 1)
            reward = 0.0 if is_done(s) else -1.0

            # We're stuck in a terminal state
            if is_done(s):
                P[s][UP] = [(1.0, s, reward, True)]
                P[s][RIGHT] = [(1.0, s, reward, True)]
                P[s][DOWN] = [(1.0, s, reward, True)]
                P[s][LEFT] = [(1.0, s, reward, True)]
            # Not a terminal state
            else:
                ns_up = s if y == 0 else s - MAX_X
                ns_right = s if x == (MAX_X - 1) else s + 1
                ns_down = s if y == (MAX_Y - 1) else s + MAX_X
                ns_left = s if x == 0 else s - 1
                P[s][UP] = [(1.0, ns_up, reward, is_done(ns_up))]
                P[s][RIGHT] = [(1.0, ns_right, reward, is_done(ns_right))]
                P[s][DOWN] = [(1.0, ns_down, reward, is_done(ns_down))]
                P[s][LEFT] = [(1.0, ns_left, reward, is_done(ns_left))]

            it.iternext()
        pp.pprint(P)

g=GridworldEnv()

{   0: {   0: [(1.0, 0, 0.0, True)],
           1: [(1.0, 0, 0.0, True)],
           2: [(1.0, 0, 0.0, True)],
           3: [(1.0, 0, 0.0, True)]},
    1: {   0: [(1.0, 1, -1.0, False)],
           1: [(1.0, 2, -1.0, False)],
           2: [(1.0, 5, -1.0, False)],
           3: [(1.0, 0, -1.0, True)]},
    2: {   0: [(1.0, 2, -1.0, False)],
           1: [(1.0, 3, -1.0, False)],
           2: [(1.0, 6, -1.0, False)],
           3: [(1.0, 1, -1.0, False)]},
    3: {   0: [(1.0, 3, -1.0, False)],
           1: [(1.0, 3, -1.0, False)],
           2: [(1.0, 7, -1.0, False)],
           3: [(1.0, 2, -1.0, False)]},
    4: {   0: [(1.0, 0, -1.0, True)],
           1: [(1.0, 5, -1.0, False)],
           2: [(1.0, 8, -1.0, False)],
           3: [(1.0, 4, -1.0, False)]},
    5: {   0: [(1.0, 1, -1.0, False)],
           1: [(1.0, 6, -1.0, False)],
           2: [(1.0, 9, -1.0, False)],
           3: [(1.0, 4, -1.0, False)]},
    6: {   0: [(1.0, 2, -1.0, False)],
           1: [(1.0, 7, -1.0,